# Example: growth of perturbations in the early universe

This example is similar to the simulations in [1909.11678](https://arxiv.org/abs/1909.11678)/[10.1103/PhysRevLett.124.061301](https://doi.org/10.1103/PhysRevLett.124.061301).

## Set up Julia environment

In [ ]:
import Pkg
Pkg.activate(mktempdir())

Pkg.add("CSV")
Pkg.add("Plots")
Pkg.add("NPZ")
Pkg.add("Unitful")

In [ ]:
using JultraDark
using Random: randn!
using Test
using NPZ
using CSV
using Plots
using Unitful

In [ ]:
Threads.nthreads()

## Units

In [ ]:
mpl = uconvert(u"kg", sqrt(Unitful.ħ * Unitful.c / Unitful.G))
Mpl = uconvert(u"kg", sqrt(Unitful.ħ * Unitful.c / 8 / π / Unitful.G))

In [ ]:
tpl = uconvert(u"s", sqrt(Unitful.ħ * Unitful.G / Unitful.c^5))

In [ ]:
lpl = uconvert(u"m", sqrt(Unitful.ħ * Unitful.G / Unitful.c^3))

In [ ]:
m = 6.35e-6Mpl

In [ ]:
hbar = Unitful.ħ
G = Unitful.G
Ωm = 1.0

In [ ]:
H0 = sqrt(1/3)m / (tpl*mpl)

In [ ]:
t_code = (3 / 8 / π * H0^2 * Ωm)^(-1/2)

In [ ]:
uconvert(u"s", t_code)

In [ ]:
l_code = (hbar / m)^(1/2) * (3 / 8 / π * H0^2 * Ωm)^(-1/4)

In [ ]:
uconvert(u"m", l_code)

In [ ]:
m_code = (hbar / m)^(3/2) * (3 / 8 / π * H0^2 * Ωm)^(+1/4) / G

In [ ]:
uconvert(u"kg", m_code)

## Define initial conditions

In [ ]:
function x(k, a)
    m = 1
    H0 = 1
    k^2 / (m * H0 * a^0.5)
end

function δ_g(x)
    if x == 0
        0
    else
        -(3/x^2 - 1) * cos(x) - 3/x * sin(x)
    end
end

function S_g(x)
    if x == 0
        0
    else
        (6/x^3 + 3/x) * cos(x) + (6/x^2 - 1) * sin(x)
    end
end

function propagate_to!(grids, A_k, a)
    # Density perturbation
    δ_k = similar(grids.ρk)
    δ_k .= A_k .* δ_g.(x.(grids.rk, a))
    δ_k[1, 1, 1] = 0

    # Phase perturbation
    S_k = similar(grids.ρk)
    S_k .= A_k .* S_g.(x.(grids.rk, a))

    # Field perturbation
    grids.ψx .= (1+0im .+ grids.rfft_plan \ δ_k).^0.5 .* exp.(im .* (grids.rfft_plan \ S_k))
    
    return
end

function propagate_to(grids, A_k, a)
    grids_new = deepcopy(grids)
    
    propagate_to!(grids_new, A_k, a)
    
    grids_new
end

In [ ]:
a_end = 1
t_end = (2.0/(3.0 * H0))
t_end_code_units = uconvert(Unitful.NoUnits, t_end/t_code)

function a(t)
    a_end * (t/t_end_code_units)^(2/3)
end

function t(a)
    t_end_code_units * (a/a_end)^(3/2)
end

In [ ]:
L_horizon_end = uconvert(Unitful.NoUnits, 1/(H0*tpl) * lpl / l_code)

In [ ]:
resol = 64
box_length = 20 * L_horizon_end

In [ ]:
# Define initial conditions

a_init = 1e0
a_final = 1e1

@show t_init = t(a_init)
@show t_final = t(a_final)

grids = JultraDark.Grids(box_length, resol)

A_k = similar(grids.ρk)
randn!(A_k)
A_k[grids.rk .> maximum(grids.rk[1, 1, :])] .= 0
A_k *= 1e-3

propagate_to!(grids, A_k, a_init)

In [ ]:
output_dir = "$(pwd())/output"
output_times = t_init:(t_final-t_init)/5:t_final
output_config = OutputConfig(output_dir, output_times)

options = Config.SimulationConfig(10, a)

## Run simulation

In [ ]:
@time simulate(grids, options, output_config)

## Check output

In [ ]:
summary = CSV.File("$(output_config.directory)/summary.csv");

In [ ]:
plot(
    summary.a, summary.δx_rms;
    legend=false,
    xlabel=raw"$a$",
    ylabel=raw"$\mathrm{rms}(\delta)$",
    xscale=:log10
)

In [ ]:
plot(
    summary.t, summary.Δt,
    legend=false,
    xlabel=raw"$t$",
    ylabel=raw"$\Delta t$",
)

In [ ]:
rho_init = npzread("$(output_config.directory)/rho_1.npy");
rho_last = npzread("$(output_config.directory)/rho_$(length(output_times)).npy");

In [ ]:
contourf(rho_init[1, :, :] .- 1; aspectratio=:equal)
xlims!(1, resol)

In [ ]:
contourf(rho_last[1, :, :] .- 1; aspectratio=:equal)
xlims!(1, resol)

In [ ]:
grids_pert = propagate_to(grids, A_k, a_init)
rho_last_pert = (abs.(grids_pert.ψx).^2)
contourf(rho_last_pert[1, :, :] .- 1; aspectratio=:equal)
xlims!(1, resol)

In [ ]:
contourf(rho_last_pert[1, :, :] .- rho_last[1, :, :]; aspectratio=:equal)
xlims!(1, resol)